# Merged Commitment Analysis (2P3G)

This notebook merges commitment analyses across Human–Human, Human–GPT, and Human–RL conditions, focusing on 2P3G.

- Human–Human and Human–GPT data: `dataAnalysis/human-human-with-gpt-fallback/oldAnalytsis/commitment_analysis_clean.ipynb` logic
- Human–RL data: `dataAnalysis/human-RLs/2-commitment_analysis.ipynb` outputs

Outputs:
- Participant-level merged table for 2P3G
- Bar plot of human commitment across four types: Human–Human, Human–GPT, RL (Individual), RL (Joint)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context('talk')

# Paths
HHGPT_DIR = Path('dataAnalysis/human-human-with-gpt-fallback/oldAnalytsis')
HRL_DIR = Path('dataAnalysis/human-RLs')

# Files
hhgpt_trials_csv = HHGPT_DIR / 'combined_experiment_data.csv'
hhgpt_class_csv = HHGPT_DIR / 'game_classification_results.csv'
hrl_participant_csv = HRL_DIR / 'commitment_analysis_participant_level_data.csv'

hhgpt_trials_csv, hhgpt_class_csv, hrl_participant_csv


## Load and prepare HH–GPT trial-level data (2P3G)
- Merge game class by file id
- Compute human commitment using shared goal vs human final goal
- Ensure human index handling is correct for Human–GPT and defaults to player1 for Human–Human (NaN index)


In [ ]:
# Load HH–GPT trials
df_hhgpt = pd.read_csv(hhgpt_trials_csv)
# Derive file_id to join classification (aligned with commitment_analysis_clean.ipynb)
df_hhgpt['file_id'] = df_hhgpt['source_file']

# Merge classification
class_map = pd.read_csv(hhgpt_class_csv)[['file_id','game_class']]
df_hhgpt = df_hhgpt.merge(class_map, on='file_id', how='left')

# Filter Human–Human and Human–GPT only
df_hhgpt = df_hhgpt[df_hhgpt['game_class'].isin(['human-human', 'human-gpt'])].copy()

# Focus on 2P3G
df_hhgpt_2p3g = df_hhgpt[df_hhgpt['experimentType'] == '2P3G'].copy()

# Standardize participant ID column name
if 'participantId' in df_hhgpt_2p3g.columns and 'participantID' not in df_hhgpt_2p3g.columns:
    df_hhgpt_2p3g = df_hhgpt_2p3g.rename(columns={'participantId':'participantID'})

len(df_hhgpt_2p3g), df_hhgpt_2p3g[['participantID','game_class']].drop_duplicates().shape[0]


In [ ]:
# Commitment functions (aligned with commitment_analysis_clean.ipynb)
def calculate_human_commitment_2p3g(row):
    shared_goal = row.get('firstDetectedSharedGoal')
    human_index = row.get('humanPlayerIndex')
    # Default to player1 for human-human (NaN human index)
    if pd.isna(human_index):
        human_index = 0
    if pd.isna(shared_goal):
        return np.nan
    # Get human player's final reached goal
    if human_index == 0:
        human_final_goal = row.get('player1FinalReachedGoal')
    else:
        human_final_goal = row.get('player2FinalReachedGoal')
    if pd.isna(human_final_goal):
        return np.nan
    try:
        return 1 if int(shared_goal) == int(human_final_goal) else 0
    except (ValueError, TypeError):
        return np.nan

# Compute human commitment for HH–GPT (2P3G)
df_hhgpt_2p3g['human_commitment'] = df_hhgpt_2p3g.apply(calculate_human_commitment_2p3g, axis=1)

# Participant-level (mean over trials)
hhgpt_participant = (
    df_hhgpt_2p3g
    .groupby(['participantID','game_class'], as_index=False)['human_commitment']
    .mean()
)
hhgpt_participant = hhgpt_participant.rename(columns={'human_commitment':'commitment_rate'})
hhgpt_participant['group_type'] = hhgpt_participant['game_class']
hhgpt_participant = hhgpt_participant[['participantID','group_type','commitment_rate']]
hhgpt_participant.head()


## Load Human–RL participant-level data (2P3G)
Map RL agent type to two additional groups: RL (Individual), RL (Joint).


In [ ]:
df_hrl = pd.read_csv(hrl_participant_csv)
df_hrl_2p3g = df_hrl[df_hrl['experiment_category'] == '2P3G'].copy()
# Map rlAgentType to unified group names
mapping = {'individual': 'human-rl-individual', 'joint': 'human-rl-joint'}
df_hrl_2p3g['group_type'] = df_hrl_2p3g['rlAgentType'].map(mapping)
df_hrl_2p3g = df_hrl_2p3g[['participantID','group_type','commitment_rate']]
df_hrl_2p3g.head()


## Merge and Plot (2P3G across four types)
Groups: Human–Human, Human–GPT, Human–RL (Individual), Human–RL (Joint).


In [ ]:
merged = pd.concat([hhgpt_participant, df_hrl_2p3g], ignore_index=True)
# Order groups
group_order = ['human-human', 'human-gpt', 'human-rl-individual', 'human-rl-joint']
merged['group_type'] = pd.Categorical(merged['group_type'], categories=group_order, ordered=True)

summary = (
    merged.groupby('group_type')['commitment_rate']
    .agg(['mean','count'])
    .rename(columns={'count':'n'})
)
summary['sem'] = merged.groupby('group_type')['commitment_rate'].sem()
summary = summary.reset_index()
summary


In [ ]:
# Plot
fig, ax = plt.subplots(figsize=(8,5))
colors = {
    'human-human': 'steelblue',
    'human-gpt': 'indianred',
    'human-rl-individual': 'darkorange',
    'human-rl-joint': 'seagreen'
}
x = np.arange(len(group_order))
means = [summary.set_index('group_type').loc[g, 'mean'] if g in summary['group_type'].values else np.nan for g in group_order]
sems = [summary.set_index('group_type').loc[g, 'sem'] if g in summary['group_type'].values else np.nan for g in group_order]
ns = [int(summary.set_index('group_type').loc[g, 'n']) if g in summary['group_type'].values else 0 for g in group_order]

bars = ax.bar(x, means, yerr=sems, capsize=5, color=[colors[g] for g in group_order], alpha=0.85)
ax.set_xticks(x)
ax.set_xticklabels(['Human–Human', 'Human–GPT', 'RL (Indiv.)', 'RL (Joint)'])
ax.set_ylim(0, 1.05)
ax.set_ylabel('Human Commitment Rate')
ax.set_title('2P3G: Commitment Across Four Game Types')
# Annotate Ns
for xi, bi, ni, mi in zip(x, bars, ns, means):
    ax.text(xi, bi.get_height() + (0.02 if not np.isnan(mi) else 0.0), f'n={ni}', ha='center', va='bottom', fontsize=10)
plt.tight_layout()
out_path = Path('dataAnalysis') / 'commitment_2p3g_four_types.png'
plt.savefig(out_path, dpi=200)
out_path
